In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../dataset')

import numpy as np
import cv2
from matplotlib import pyplot as plt
import itertools
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter
import numpy as np
from coin_detector import crop_coin
import pandas as pd
from utils import get_filename
from PIL import Image
from IPython.display import clear_output

# Circles

In [ ]:
# name = "niemczyk"
name = "marciniak"
keyword = "Sztandar 1930"

root = "../dataset/{}/img/sztandar_1930".format(name, get_filename(keyword))

In [ ]:
df = pd.read_csv(root + "/labels.csv")
for name in df[df["label"] > 0]["name"]:
    try:
        img = cv2.imread(root + "/" + name)
        crop = crop_coin(img)
        cv2.imwrite(root + "/cropped/" + name, crop)
    except:
        print('error')
        pass

## Investigating of results

### False circle

In [ ]:
name = './124_1.jpg'

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 8))

img_orig = cv2.imread(root + "/" + name)
im_1 = crop_coin(img_orig)
im_2 = im_1[10:, 10:]
im_2 = crop_coin(im_2)

ax1.imshow(im_1)
ax2.imshow(im_2)
plt.show()

### Individual mask

In [ ]:
name = './1_2.jpg'

fig, ax = plt.subplots(figsize=(8, 8))

im = cv2.imread(root + "/" + name)
im = crop_coin(im)

ax.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.show()

## Fixing

In [ ]:
def validate_masks(path, csv_out):
    labels = []
    path = path + '/'
    img_list = sorted(os.listdir(path))
    
    try:
        df = pd.read_csv(csv_out)
    except:
        df = pd.DataFrame({"name": img_list, "valid": None})

    for i, row in df.iterrows():
        if pd.isna(row["valid"]):
            im = Image.open(path + row["name"])
            plt.figure(figsize=(8,8))
            plt.imshow(im)
            plt.axis('off') 
            plt.title(row["name"])
            plt.show()
            inp = input()
            if inp == "":
                inp = 1
            else:
                inp = 0

            df["valid"][i] = inp
            df.to_csv(csv_out, index=None)
            clear_output(wait=True)
      
    return (df)

In [ ]:
df = validate_masks(root + "/cropped", root + "/cropped/" + "valid.csv")
df[df["valid"] != 0] 

# Ellipses

In [ ]:
name = './339_2.jpg'
height_img_ds = 500

img_orig = cv2.imread(name)

ds_ratio = height_img_ds/img_orig.shape[0] 
img_ds = cv2.resize(img_orig, None, fx=ds_ratio, fy=ds_ratio, interpolation=cv2.INTER_AREA) # downsampling

gray = cv2.cvtColor(img_ds, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)

thresh = cv2.Canny(gray, 50, 100)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (11, 11))
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

_, th = cv2.threshold(closed, 1, 1, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(th, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
max_contour = max(contours, key=cv2.contourArea)
img_max_contour = cv2.drawContours(np.zeros(gray.shape), max_contour, -1, (255,255,255), 1)

elipses = hough_ellipse(img_max_contour, accuracy=30, threshold=100, min_size=int(0.4 * closed.shape[0]))
elipses.sort(order='accumulator')
best_ellipse = list(elipses[-1])
(yc, xc, a, b), orientation = [int(round(x/ds_ratio)) for x in best_ellipse[1:5]], best_ellipse[5]
cy, cx = ellipse_perimeter(yc, xc, a, b, orientation)
img_orig[cy, cx] = (250, 0, 0)

fig, ax = plt.subplots( figsize=(8, 8))
ax.imshow(img_orig)
plt.show()